In [1]:
# Python core
from typing import Optional, Callable, TypedDict, Union, Iterable, Tuple, NamedTuple, List
from dataclasses import dataclass
import datetime
from itertools import product
from concurrent import futures

# Scientific python
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

# PyTorch
# import torch
# from torch import nn
# import torch.nn.functional as F
# from torchvision import transforms
# import pytorch_lightning as pl

In [2]:
import bokeh
print('bokeh version: ', bokeh.__version__)

bokeh version:  3.2.1


In [13]:
if 'client' in locals():
    client.shutdown()
    print('...shutdown client...')
else:
    print('client does not exist yet')

...shutdown client...


In [4]:
from distributed import Client
from dask_jobqueue import PBSCluster

cluster = PBSCluster(account='NAML0001',walltime='12:00:00',cores=1, memory='100GB',shared_temp_directory='/glade/scratch/wchapman/tmp',queue='casper',interface='ext')
cluster.scale(jobs=40)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/wchapman/Dingo/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/wchapman/Dingo/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.63:36725,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/wchapman/Dingo/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Load the Zarr file

In [3]:
%%time
ZARR = '/glade/derecho/scratch/wchapman/STAGING/All_2010_staged.zarr'

plt.rcParams['figure.figsize'] = (5, 5)
plt.rcParams['image.interpolation'] = 'none'

def get_forward_data(filename: str=ZARR) -> xr.DataArray:
    """Lazily opens the Zarr store on gladefilesystem.
    """
    dataset = xr.open_zarr(filename, consolidated=True)
    return dataset

forcing_data = get_forward_data().unify_chunks()
forcing_data

CPU times: user 2.53 s, sys: 237 ms, total: 2.77 s
Wall time: 27 s


<xarray.Dataset>
Dimensions:     (time: 8808, level: 15, latitude: 640, longitude: 1280,
                 half_level: 138)
Coordinates:
  * half_level  (half_level) int32 1 2 3 4 5 6 7 ... 132 133 134 135 136 137 138
  * latitude    (latitude) float64 89.78 89.51 89.23 ... -89.23 -89.51 -89.78
  * level       (level) int32 10 30 40 50 60 70 80 90 95 100 105 110 120 130 136
  * longitude   (longitude) float64 0.0 0.2812 0.5625 ... 359.2 359.4 359.7
  * time        (time) datetime64[ns] 2010-01-01 ... 2011-01-02T23:00:00
Data variables:
    Q           (time, level, latitude, longitude) float32 dask.array<chunksize=(10, 4, 214, 427), meta=np.ndarray>
    SP          (time, latitude, longitude) float32 dask.array<chunksize=(10, 214, 427), meta=np.ndarray>
    T           (time, level, latitude, longitude) float32 dask.array<chunksize=(10, 4, 214, 427), meta=np.ndarray>
    U           (time, level, latitude, longitude) float32 dask.array<chunksize=(10, 4, 214, 427), meta=np.ndarray>
    V           (time, level, latitude, longitude) float32 dask.array<chunksize=(10, 4, 214, 427), meta=np.ndarray>
Attributes: (12/18)
    COMPUTATIONAL_PLATFORM:                 cheyenne supercomputer, NCAR-Wyom...
    COMPUTATIONAL_SPECIALIST:               Dave Stepaniak, NCAR/CISL/DECS, d...
    CONVERSION_AND_COMPRESSION_PLATFORM:    cheyenne node employed: Linux r2i...
    CONVERSION_AND_COMPRESSION_SPECIALIST:  Dave Stepaniak, NCAR/CISL/DECS, d...
    CONVERSION_AND_COMPRSSION_DATE:         Sat Dec 24 17:14:38 MST 2022
    Conventions:                            CF-1.6
    ...                                     ...
    NETCDF_VERSION:                         4.8.1
    RDA_DATASET:                            ds633.6
    RDA_DATASET_DOI:                        DOI: 10.5065/XV5R-5344
    RDA_DATASET_GROUP:                      ERA5 atmospheric model level anal...
    RDA_DATASET_URL:                        https:/rda.ucar.edu/datasets/ds63...
    history:                                Sat Dec 24 17:14:51 2022: ncks -4...

## Take a Variety of Means/STDs and Save them
- you need a massive amount of compute to do this; try it in the morning before every other NCAR employee wakes up 

In [11]:
%%time
Mean_latlonlev = forcing_data.mean(['time','latitude','longitude','level']).persist()
print('...moving on to load...')
Mean_latlonlev = Mean_latlonlev.load()
print('...mean done...')
print('...saving...')
Mean_latlonlev.to_netcdf('/glade/derecho/scratch/wchapman/STAGING/All_2010_staged.mean.nc')


/glade/work/wchapman/miniconda3.1/envs/MLWPS/lib/python3.11/site-packages/distributed/client.py:3125: UserWarning: Sending large graph of size 50.19 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


...moving on to load...


...mean done...
...saving...
CPU times: user 10min 54s, sys: 20.4 s, total: 11min 14s
Wall time: 29min 39s


In [12]:
STD_latlonlev = forcing_data.std(['time','latitude','longitude','level']).persist()
# print('...moving on to load...')
STD_latlonlev = STD_latlonlev.load()
print('...saving...')
STD_latlonlev.to_netcdf('/glade/derecho/scratch/wchapman/STAGING/All_2010_staged.std.nc')

/glade/work/wchapman/miniconda3.1/envs/MLWPS/lib/python3.11/site-packages/distributed/client.py:3125: UserWarning: Sending large graph of size 50.54 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


...moving on to load...


...saving...


## Testing Below here

In [5]:
DS = xr.open_dataset('/glade/campaign/collections/rda/data/ds633.5/e5p.moda.an.pl/1950/e5p.moda.an.pl.128_131_u.ll025uv.1950010100_1950120100.nc')

In [6]:
DS

<xarray.Dataset>
Dimensions:    (time: 12, level: 37, latitude: 721, longitude: 1440)
Coordinates:
  * latitude   (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * level      (level) float64 1.0 2.0 3.0 5.0 7.0 ... 925.0 950.0 975.0 1e+03
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * time       (time) datetime64[ns] 1950-01-01 1950-02-01 ... 1950-12-01
Data variables:
    U          (time, level, latitude, longitude) float32 ...
    utc_date   (time) int32 ...
Attributes:
    DATA_SOURCE:          ECMWF: https://cds.climate.copernicus.eu, Copernicu...
    NETCDF_CONVERSION:    CISL RDA: Conversion from ECMWF GRIB 1 data to netC...
    NETCDF_VERSION:       4.8.1
    CONVERSION_PLATFORM:  Linux r3i1n15 4.12.14-95.51-default #1 SMP Fri Apr ...
    CONVERSION_DATE:      Tue Mar  1 15:58:05 MST 2022
    Conventions:          CF-1.6
    NETCDF_COMPRESSION:   NCO: Precision-preserving compression to netCDF4/HD...
    history:              Tue Mar  1 15:58:14 2022: ncks -4 --ppc default=7 e...
    NCO:                  netCDF Operators version 4.9.5 (Homepage = http://n...